In [1]:
%pip install -e git+https://github.com/saumya06/crewAI_nlsql.git@nlsql#egg=crewai
%cd '/home/wsuser/work/src/crewai'
%pip install .
%cd '/home/wsuser/work/'

Obtaining crewai from git+https://github.com/saumya06/crewAI_nlsql.git@nlsql#egg=crewai
  Updating ./src/crewai clone (to revision nlsql)
  Running command git fetch -q --tags
  Running command git reset --hard -q f0855c76257f5a048f223f237495da7ef389465c
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for crewai (pyproject.toml) ... done
  Created wheel for crewai: filename=crewai-0.108.0-py3-none-any.whl size=13487 sha256=c8e3ef3d376bdcf3e6d857a6ec2525bff0244271627b722d5f947941c86eb909
  Stored in directory: /tmp/wsuser/pip-ephem-wheel-cache-yb7wvfls/wheels/67/83/0d/e3b4d3b1ad8b43c22af6bd46f5358ab90c520863e25c50f2ae
Successfully built crewai
  Attempting uninstall: crewai
    Found existing installation: crewai 0.108.0
    Uninstalling crewai-0.108.0:
      Successf

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Processing /home/wsuser/work/src/crewai
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for crewai: filename=crewai-0.108.0-py3-none-any.whl size=265353 sha256=34e72f6197989ba68d1c3d25d3b17fd7a14b4fcce455e8168361b030e174c1f9
  Stored in directory: /tmp/wsuser/pip-ephem-wheel-cache-csmunnkt/wheels/45/0f/f7/41283362c60b2f77a5d953e3289bbeaa6bab5c024dbea49c79
Successfully built crewai
  Attempting uninstall: crewai
    Found existing installation: crewai 0.108.0
    Uninstalling crewai-0.108.0:
      Successfully uninstalled crewai-0.108.0
Note: you may need to restart the kernel to use updated packages.
/home/wsuser/work


In [2]:
%pip install crewai[tools] 
from crewai_tools import NL2SQLTool
from crewai import LLM
import os
from crewai import Agent, Crew, Task, Process

Note: you may need to restart the kernel to use updated packages.


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:404: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.


In [ ]:
DB_URL = "cb0cfae7-372d-4047-9bf9-882289cd9910.blijti4d0v0nkr55oei0.databases.appdomain.cloud"
DB_PORT = "32240"
DB_NAME = "marcht3db"
DB_USER = "ibm_cloud_d2c0138f_5cb3_499e_8043_6a959fb57913"
DB_PASSWORD = input("Enter password for the user:") 

In [14]:
nl2sql = NL2SQLTool(db_uri=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}:{DB_PORT}/{DB_NAME}")

In [ ]:

# Initialize LLM
llm_db = LLM(
    api_key="YOUR_API_KEY",
    api_base = "https://us-south.ml.cloud.ibm.com",
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 15000,
        "temperature": 0,
        "repetition_penalty": 1.05
    }
)

# Initialize LLM
llm_ba = LLM(
    api_key="YOUR_API_KEY",
    api_base = "https://us-south.ml.cloud.ibm.com",
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "sample",
        "max_new_tokens": 15000,
        "temperature": 0.8,
        "repetition_penalty": 0.5
    }
)

def create_dbagent() -> Agent:
    return Agent(
        role="Database Analyst",
        goal="""Generate the report based on user question and information in database. Always write the colum name as table_name."column_name" in sql query.""",
        backstory="""You are an expert data analyst creating reports from database.""",
        llm=llm_db,
        allow_delegation=False,
        tools=[nl2sql]
    )

def create_researcher() -> Agent:
    return Agent(
        role="Business Analyst",
        goal="""Analyse the data and user question to form the report. If asked about sales, get and analyze information such as monthly quantity, value and percentage chage, peak month, average shipping times.
        If time or trend analysis question is asked, be aware of the chronological order of the data""",
        backstory="""You are an expert data analyst creating reports from database.""",
        llm=llm_ba,
        allow_delegation=True,
    )

def create_generation_task(agent, natural_query):
    return Task(
        description=f"""{natural_query}.""",
        expected_output="""Response from database after running the query.""",
        agent=agent
    )

def create_analysis_task(agent, natural_query):
    return Task(
        description=f"""{natural_query}.""",
        expected_output="""Table or textual analysis report from data.""",
        agent=agent
    )

In [11]:
research_agent=create_researcher()
query_agent=create_dbagent()
# generation_task = create_generation_task(research_agent, "how have the sales of Xtralife changed over the past 12 months?")
analysis_task = create_analysis_task(research_agent, "how have the sales of Xtralife product over the past 12 months?") #, you can also include "from March 2024 to February 2025" in the prompt
generation_crew = Crew(
    agents=[research_agent, query_agent],
    tasks=[analysis_task],
    verbose=True
    )

In [12]:
crew_output = generation_crew.kickoff()

# Accessing the crew output
print(f"Raw Output: {crew_output.raw}")
if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")
if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")
print(f"Tasks Output: {crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b4ca0549-92ab-4eca-a26b-350b5f45c270                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4d3bae85-cc7d-44c7-a17d-4708e443919e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4d3bae85-cc7d-44c7-a17d-4708e443919e
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress

# Agent: Business Analyst
## Task: how have the sales of Xtralife product over the past 12 months?.


🚀 Crew: crew
└── 📋 Task: 4d3bae85-cc7d-44c7-a17d-4708e443919e
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: 4d3bae85-cc7d-44c7-a17d-4708e443919e
       Status: Executing Task...
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (2)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (13)

🚀 Crew: crew
└── 📋 Task: 4d3bae85-cc7d-44c7-a17d-4708e443919e
       Assigned to: Business Analyst
       Status: ✅ Completed
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (2)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (13)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4d3bae85-cc7d-44c7-a17d-4708e443919e                                                                     │
│  Agent: Business Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b4ca0549-92ab-4eca-a26b-350b5f45c270                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Raw Output: ### Sales Analysis Report for Xtralife Product

#### Introduction
This report provides an analysis of the sales of the Xtralife product over the past 12 months. The analysis includes monthly quantity, value, and percentage change, peak month, and a brief note on average shipping times.

#### Monthly Sales Breakdown

| Month | Units Sold | Sales Value |
| --- | --- | --- |
| April 2024 | 1,100 | $200,000 |
| May 2024 | 1,300 | $230,000 |
| June 2024 | 1,600 | $260,000 |
| July 2024 | 1,900 | $280,000 |
| August 2024 | 2,100 | $310,000 |
| September 2024 | 2,300 | $330,000 |
| October 2024 | 2,500 | $360,000 |
| November 2024 | 2,600 | $390,000 |
| December 2024 | 2,900 | $400,000 |
| January 2025 | 700 | $110,000 |
| February 2025 | 800 | $140,000 |

#### Key Findings

- **Total Sales**: Over the 12 months, a total of 18,300 units were sold, with a total sales value of $2,920,000.
- **Peak Sales Month**: The peak month for sales was December 2024, with 2,900 units sold and a

### Running both Analysis task and Generation task

In [15]:
research_agent=create_researcher()
query_agent=create_dbagent()
generation_task = create_generation_task(research_agent, "how have the sales of Xtralife changed over the past year")
analysis_task = create_analysis_task(research_agent, "how have the sales of Xtralife product changed over the past year")
generation_crew = Crew(
    agents=[research_agent, query_agent],
    tasks=[analysis_task,generation_task],
    verbose=True
    )

In [16]:
crew_output = generation_crew.kickoff()

# Accessing the crew output
print(f"Raw Output: {crew_output.raw}")
if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")
if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")
print(f"Tasks Output: {crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 76297bdc-0337-410c-b7fd-7466be9620a1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 78df3556-3a31-44e2-a6ca-dab8ed0af2a4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 78df3556-3a31-44e2-a6ca-dab8ed0af2a4
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress

# Agent: Business Analyst
## Task: how have the sales of Xtralife product changed over the past year.


🚀 Crew: crew
└── 📋 Task: 78df3556-3a31-44e2-a6ca-dab8ed0af2a4
       Status: Executing Task...
    └── 🤖 Agent: Business Analyst
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 78df3556-3a31-44e2-a6ca-dab8ed0af2a4
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (3)
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used NL2SQLTool (17)
│   │   └── 🔧 Using Delegate work to coworker (3)
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
│       └── 🔧 Used NL2SQLTool (20)
└── 📋 Task: 0b12f963-bae5-4757-b63d-d5d53b993163
       Status: Executing Task...
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    │   └── 🔧 Using Ask question to coworker (4)
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    │   ├── 🔧 Used NL2SQLTool (26)
    │   └── 🔧 Using Delegate work to coworker (4)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (31)

🚀 Crew: crew
├── 📋 Task: 78df3556-3a31-44e2-a6ca-dab8ed0af2a4
│      Assigned to: Business Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Business Analyst
│   │       Status: In Progress
│   │   └── 🔧 Using Ask question to coworker (3)
│   ├── 🤖 Agent: Database Analyst
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used NL2SQLTool (17)
│   │   └── 🔧 Using Delegate work to coworker (3)
│   └── 🤖 Agent: Business Analyst
│           Status: ✅ Completed
│       └── 🔧 Used NL2SQLTool (20)
└── 📋 Task: 0b12f963-bae5-4757-b63d-d5d53b993163
       Assigned to: Business Analyst
       Status: ✅ Completed
    ├── 🤖 Agent: Business Analyst
    │       Status: In Progress
    │   └── 🔧 Using Ask question to coworker (4)
    ├── 🤖 Agent: Database Analyst
    │       Status: ✅ Completed
    │   ├── 🔧 Used NL2SQLTool (26)
    │   └── 🔧 Using Delegate work to coworker (4)
    └── 🤖 Agent: Business Analyst
            Status: ✅ Completed
        └── 🔧 Used NL2SQLTool (31)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0b12f963-bae5-4757-b63d-d5d53b993163                                                                     │
│  Agent: Business Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 76297bdc-0337-410c-b7fd-7466be9620a1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Raw Output: Monthly Sales Report for Xtralife Product

| Month | Quantity Sold | Total Sales | Percentage Change |
| --- | --- | --- | --- |
| March 2024 | 700 | 110,000 | No previous month to compare |
| April 2024 | 1100 | 200,000 | 5.26% increase from March |
| May 2024 | 1300 | 230,000 | 15.00% increase from April |
| June 2024 | 1600 | 260,000 | 13.04% increase from May |
| July 2024 | 1900 | 280,000 | 7.69% increase from June |
| August 2024 | 2100 | 310,000 | 10.71% increase from July |
| September 2024 | 2300 | 330,000 | 6.45% increase from August |
| October 2024 | 2500 | 360,000 | 9.09% increase from September |
| November 2024 | 2600 | 390,000 | 8.33% increase from October |
| December 2024 | 2900 | 400,000 | 2.56% increase from November |
| January 2025 | 700 | 110,000 | -72.50% decrease from December |
| February 2025 | 800 | 140,000 | 27.27% increase from January |

The total sales for the past 12 months are approximately 2,820,000. The quantity sold ranges from 700 to 2,